# Import Packages

In [13]:
library(RColorBrewer)
library(reshape)
library(gganimate)
library(tidyverse)
library(data.table)

# Read And Prepare Data

In [3]:
Data <- read.csv("FootballData.csv", fileEncoding = "ISO-8859-1")
names(Data) <- c("Club","Income","Country","Year","Position","Colour")
Data$ClubYear <- paste(Data$Club, Data$Year)
Data$temp <-  paste(Data$Club,Data$Country,Data$Colour, Data$Year)
Data <- Data[order(Data$Year, decreasing = FALSE),]
summary(Data)

                Club         Income          Country         Year     
 Real Madrid      : 37   Min.   : 83.1   England :182   Min.   :1955  
 Bayern Munich    : 30   1st Qu.:129.4   Italy   :111   1st Qu.:1992  
 Milan            : 28   Median :172.9   Spain   : 86   Median :2009  
 Juventus         : 27   Mean   :223.7   Germany : 85   Mean   :2002  
 Manchester United: 25   3rd Qu.:263.5   France  : 41   3rd Qu.:2014  
 Barcelona        : 24   Max.   :750.9   Portugal: 20   Max.   :2019  
 (Other)          :435   NA's   :193     (Other) : 81                 
    Position         Colour      ClubYear             temp          
 Min.   :1.000   #FEBE10: 37   Length:606         Length:606        
 1st Qu.:1.000   #000000: 33   Class :character   Class :character  
 Median :2.500   #DC052D: 30   Mode  :character   Mode  :character  
 Mean   :2.246   #FB090B: 28                                        
 3rd Qu.:3.000   #DA020E: 25                                        
 Max.   :3.000   #

In [4]:
Dataset <- Data
Dataset <- unique.data.frame(Data[c("Club","Country","Colour")])
Dataset[as.character(unique(Data$Year))] <- 0
Dataset <- melt(Dataset, id = c("Club","Country","Colour"))
names(Dataset)[4:5] <- c("Year","Position")
Dataset$temp <- paste(Dataset$Club,Dataset$Country,Dataset$Colour,Dataset$Year)
Dataset$Position <- Data$Position[match(Dataset$temp, Data$temp)]
head(Dataset,20)

#for (i in 1:nrow(Dataset)){
#    temp <- Dataset[1:i,]
#    temp <- sum(temp$InCriteria[temp$Club==Dataset$Club[i]], na.rm = TRUE)
#    Dataset$Sum[i] <- as.numeric(temp)
#    }
#Dataset <- Dataset %>%
#    group_by(Year) %>%
#    mutate(Rank = order(order(Sum, Club, decreasing=TRUE)))
#Dataset <- Dataset[Dataset$Rank < 10,]
#head(Dataset,50)

Club,Country,Colour,Year,Position,temp
<fct>,<fct>,<fct>,<fct>,<int>,<chr>
Milan,Italy,#FB090B,1955,3,Milan Italy #FB090B 1955
Real Madrid,Spain,#FEBE10,1955,1,Real Madrid Spain #FEBE10 1955
Hibernian,Scotland,#00753b,1955,3,Hibernian Scotland #00753b 1955
Reims,France,#d2232a,1955,2,Reims France #d2232a 1955
Manchester United,England,#DA020E,1955,NA,Manchester United England #DA020E 1955
Fiorentina,Italy,#EC1C23,1955,NA,Fiorentina Italy #EC1C23 1955
Crvena zvezda,Serbia,#d90512,1955,NA,Crvena zvezda Serbia #d90512 1955
Vasas,Hungary,#265eb4,1955,NA,Vasas Hungary #265eb4 1955
Atlético Madrid,Spain,#272E61,1955,NA,Atlético Madrid Spain #272E61 1955


# Define Function For Creating Ranked Datasets

In [5]:
PrepareData <- function(Dataset, PositionNumber){
    Dataset$InCriteria <- ifelse(Dataset$Position <= PositionNumber, 1,0)
    Dataset$Sum <- NA
    for (i in 1:nrow(Dataset)){
        temp <- Dataset[1:i,]
        temp <- sum(temp$InCriteria[temp$Club==Dataset$Club[i]], na.rm = TRUE)
        Dataset$Sum[i] <- as.numeric(temp)
        }
    Dataset <- Dataset %>%
        group_by(Year) %>%
        mutate(Rank = order(order(Sum, Club, decreasing=TRUE)))
    Dataset <- Dataset[Dataset$Rank < 20 & Dataset$Sum >0,]
    return(Dataset)
    }

# Define Function To Create Colour Scheme

# Run Function To Produce Data For Winners, Reached Finals, Semis

In [6]:
FirstPlaceData <- PrepareData(Dataset, 1)
FinalsData <- PrepareData(Dataset, 2)
SemisData <- PrepareData(Dataset, 3)

# Define Function For Animated Graphs Showing Rank

In [10]:
CreateGraph <- function(Dataset){
    ggplot(Dataset, aes(Rank, group = Club))+
    geom_col(aes(y=Sum, color = Colour, fill = Club))+
    coord_flip(clip = "off", expand = FALSE) +
      scale_x_reverse() +
      guides(color = FALSE, fill = FALSE) + #Add Watermark With Website Name
      annotate(
        "text",
        x = -0.5,
        y = 0,
        label = "www.TheDataInspector.com",
        #vjust = -5.5,
          hjust = -1.5,
        colour = "#8D8D8D",
        size = 6
      ) +
      geom_text(
        aes(
          y = 0,
          label = paste(Club, " "),
            color = Club
        ),
        vjust = 0.2,
        hjust = 1,
        size = 6
      ) + 
    geom_text(
        aes(
          #y = max(Income) * 0.2,
          y = 0,
            x = -0.5,
          label = as.character(Year),
          fontface = 2,
            color = Club
        ),
        size = 12,
        #vjust = -2.5,
        colour = "#8D8D8D",
        check_overlap = TRUE
      ) +
     geom_text(aes(y = Sum, label = as.character(Sum)),
                size = 8,
                nudge_y = 10) +
      labs(title = paste("Champions League Ranking"))+
      theme(
        #Set Theme Options
        axis.line = element_blank(),
        axis.text.y = element_blank(),
        axis.text.x = element_blank(),
        axis.ticks = element_blank(),
        axis.title = element_blank(),
        legend.position = "none",
        panel.background = element_blank(),
        panel.border = element_blank(),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        panel.grid.major.x = element_line(size = .05, colour = "#DBDBDB"),
        panel.grid.minor.x = element_line(size = .05, colour = "#DBDBDB"),
        plot.title = element_text(
          size = 30,
          hjust = 0.5,
          face = "bold",
          colour = "#8D8D8D",
          vjust = 5
        ),
        plot.background = element_blank(),
        plot.margin = margin(3, 2, 2, 10, "cm")
      ) + #Add Transition State Controlling Time Spent Moving And Time Spent At Data Point
      transition_states(Year, transition_length = 10, state_length = 10) + #Control If X & Y Axis Should Scale As Data Changes
      view_follow(fixed_x = TRUE, fixed_y = TRUE)
    }

# Run Function With Different Datasets

In [11]:
animate(CreateGraph(FirstPlaceData), height = 750, width = 900, nframes = 200, fps = 40, renderer = gifski_renderer("FirstPlace.gif"))
#animate(CreateGraph(FinalsData), height = 750, width = 900, nframes =2000, fps = 40, renderer = gifski_renderer("Finals.gif"))
#animate(CreateGraph(SemisData), height = 750, width = 900, nframes = 2000, fps = 40, renderer = gifski_renderer("Semis.gif"))